In [ ]:
#add comments
from transformers import pipeline #this is the pipeline module from the transformers library
from transformers import  TFAutoModelForSequenceClassification, AutoTokenizer #this is the model and tokenizer module from the transformers library
import pandas as pd #this is the pandas library
import numpy as np #this is the numpy library
import tensorflow as tf #this is the tensorflow library

C:\Users\alper\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_name = "bert-tr-sentiment" #this is the model name
model = TFAutoModelForSequenceClassification.from_pretrained(model_name) #this is the model
tokenizer = AutoTokenizer.from_pretrained(model_name) #this is the tokenizer

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
file = "data/istenen1.csv"  #this is the file
df = pd.read_csv(file) #this is the dataframe

In [ ]:
def process_comments(filename, column_name):  #this is the function
    # Load the data
    df = pd.read_csv(filename)
    
    # Truncate the input text to the maximum token length supported by the model
    def predict(text):
        inputs = tokenizer(text, return_tensors="tf", truncation=True, max_length=512)
        output = model(**inputs)
        return output.logits.numpy()

    # Use the model to predict the sentiment of the column of comments and create a new column with the predictions
    df["sentiment"] = df[column_name].apply(lambda x: predict(x))
    df["sentiment"] = df["sentiment"].apply(lambda x: np.argmax(x, axis=1))
    df["sentiment"] = df["sentiment"].apply(lambda x: "positive" if x == 1 else ("negative" if x == -1 else "neutral"))
    
    # Making a new table with the sentiment values and the comments
    df = df[["sentiment", column_name]]
    
    # Save the processed dataframe to a new CSV file
    output_filename = filename.replace(".csv", "_sentiment.csv")
    df.to_csv(output_filename, index=False)
    
    return df



,sentiment,Comments
0,neutral,hastalarına ne zaman geleceği bellı olmayan ve...
1,positive,"tedavi shekli, hastalıgın seyrıne gore gun ash..."
2,positive,turkiyede yaklashık 30.000mshastası bulunmakta...
3,positive,bu da birautoimmune diseasedir.
4,neutral,merkez sinir sisteminin en sık görülen hastalı...


In [20]:
data2 = pd.read_csv("data/istenen2.csv")
data2 = data2[:50]
data2 = data2["Comments"]
data2.head()

data2.to_csv("datatable.csv", index=False)

In [23]:

import tkinter as tk
from tkinter import filedialog, messagebox

def browse_file():
    file_path = filedialog.askopenfilename(title="Dosya Seç")
    if file_path:
        file_entry.delete(0, tk.END)
        file_entry.insert(0, file_path)

def submit():
    file_path = file_entry.get()
    column_name = column_entry.get()

    if not file_path or not column_name:
        messagebox.showerror("Hata", "Lütfen dosya konumunu ve sütun ismini giriniz!")
    else:
        # Burada sentiment analiz fonksiyonunuzu çağırabilirsiniz
        print(f"Dosya Konumu: {file_path}")
        print(f"Sütun İsmi: {column_name}")
        df = process_comments(file_path, column_name)

        #when the process is done notify the user
        messagebox.showinfo("Bilgi", "Sentiment analiz işlemi tamamlandı!")
        #dosyanın konumuna erişimi sağla
        print(f"Sentiment analiz sonucu: {file_path.replace('.csv', '_sentiment.csv')}")

# Ana pencere
window = tk.Tk()
window.title("Sentiment Analiz Arayüzü")

# Dosya Konumu
tk.Label(window, text="Dosya Konumu:").grid(row=0, column=0, padx=10, pady=10)
file_entry = tk.Entry(window, width=50)
file_entry.grid(row=0, column=1, padx=10, pady=10)
browse_button = tk.Button(window, text="Gözat", command=browse_file)
browse_button.grid(row=0, column=2, padx=10, pady=10)

# Sütun İsmi
tk.Label(window, text="Sütun İsmi:").grid(row=1, column=0, padx=10, pady=10)
column_entry = tk.Entry(window, width=50)
column_entry.grid(row=1, column=1, padx=10, pady=10)

# Gönder butonu
submit_button = tk.Button(window, text="Gönder", command=submit)
submit_button.grid(row=2, column=1, padx=10, pady=10)

# Pencereyi başlat
window.mainloop()


df.head()


,sentiment,Comments
0,neutral,hastalarına ne zaman geleceği bellı olmayan ve...
1,positive,"tedavi shekli, hastalıgın seyrıne gore gun ash..."
2,positive,turkiyede yaklashık 30.000mshastası bulunmakta...
3,positive,bu da birautoimmune diseasedir.
4,neutral,merkez sinir sisteminin en sık görülen hastalı...
